
https://github.com/FudanNLP/nlp-beginner 参考邱锡鹏老师的task

https://nndl.github.io/nndl-book.pdf 《神经网络与深度学习》邱锡鹏

In [1]:
#线性回归
#tensorflow做梯度下降回归
import tensorflow.compat.v1 as tf#tensorflow1.0版本
tf.disable_eager_execution()

class LrModel():
    def __init__(self, config, seq_length):
        self.config = config
        self.seq_length = seq_length
        self.lr()

    def lr(self):
        self.x = tf.placeholder(tf.float32, [None, self.seq_length])
        w = tf.Variable(tf.zeros([self.seq_length, self.config.num_classes]))
        b = tf.Variable(tf.zeros([self.config.num_classes]))

        y = (tf.matmul(self.x, w) + b)

        self.y_pred_cls = tf.argmax(y, 1)

        self.y_ = tf.placeholder(tf.float32, [None, self.config.num_classes])
        cross_entropy = tf.reduce_mean(-tf.reduce_sum(self.y_ * tf.log(y), reduction_indices=[1]))
        self.loss = tf.reduce_mean(cross_entropy)

        self.train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)

        correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(self.y_, 1))
        self.accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))


# 文本分类任务：

数据集用 $X$ 表示,$X = {(x^1 , y^1 ), ... , (x^N , y^N )}$,N 为 样本个数。$y^i$ 为样本实例 $x^i$ 的类别标记。在自然语言处理中,数据集也经常称为语料库。

## 向量化:
在机器学习中,为了更好地表示样本的属性,一般将样本表示成代数形式,称为样本的特征,我们用$\phi(x)$。样本特征可以是一维或多维向量,$\phi(x) ∈ R_m,m$是向量维数。 $$ \phi(x) = \left( \begin{array} {ccc} \phi_1(x) \ \phi_2 (x) \ .\ .\ \phi_m(x) \end{array} \right) $$自然语言处理中,数据都是以字符形式存在的。
样本的原始表示一般是字符串序列。为了便于使用机器学习方法,首先要把样本表示为向量形式。下面我们介绍几种常 用的特征表示方法。自然语言处理的,在构造了样本和样本集合之后,为了和后面的机 器学习算法相结合,我们将样本x 转变成向量φ(x)。在将字符表示转换成向量表示的过 程中需要很多中间步骤,我们把这些中间步骤都成为数据处理,并且尽可能的模块化。
>> 2.1 词袋模型
一种简单的方法是简单假设文本(如一个句子或一个文档)是由字、词组成的无序多重集合,不考虑语法甚至词序。这就是在自然语言处理和信息检索中常用的词袋模型，词袋模型可以看成一种以词为基本单位的向量空间模型(Vector Space Model, VSM)。具体可见本课程chap3的slide

>>2.2 N 元特征
词袋模型在需要深层分析的场合就会显得太过简化了。例如在语义分析里,“你打了 我”和“我打了你”,意思是相反的,但用词袋模型表示后,这两句话是向量表示的等价 的,这显然是不合理的。

>>N 元特征(N-gram 特征),顾名思义,就是由 N 个字或词组成的字符串,单元可以 是字或词。这里N是大于等于1的任意整数。如果N 为2,就叫做二元特征,如果N为 3,就叫做三元特征以此类推。

>>N 元特征可以看出是对词袋模型的一种改进方法。与 N 元特征相关的概念是 N 元语法模型。以中文句子“机器学习算法”为例,以字为基本单位的二元特征集合为:{机器,器 学,学习,习算,算法}。集合中每一项都是由二个相邻的字组成的的子串,长度为 2。这 些子串可以是有意义的词(例如:“学习”、“算法”),也可以是无任何意义的字符串(例 如:“器学”,“习算”)。但是这些无意义的子串也有可能在分类中起到很关键的作用。一 个长度为L的句子,可以提取出L − 1个二元特征。

>>有了 N 元特征集合,就可以利用词袋模型将文本表示为向量形式。随着 N 的增加, 可以抽取的特征就会越多,特征空间也会呈指数增加。这些高阶的特征出现的频率也会相对较低,对分类不但没有太多帮助,还会直接影响着后续处理的效率与复杂度。因此在一般的文本分类任务中,N 取 3 就足够了,并且同时也使用一元和二元特征,防止出现过拟合。

# 文本分类的做法
> 经过特征抽取后,一个样本可以表示为 k 维特征空间中的一个点。为了对这个特征空间中的点进行区分,就需要寻找一些超平面来将这个特征空间分为一些互不重叠的子区域,使得不同类别的点分布在不同的子区域中,这些超平面就成为判别界面。
为了定义这些用来进行空间分割的超平面,就需要引入判别函数的概念。假设变量 $z ∈ R_m$为特征空间中的点,这个超平面由所有满足函数f(z) = 0的点组成。这里的f(z)就称为判别函数。
有了判别函数,分类就变得很简单,就是看一个样本在特征空间中位于哪个区域, 从而确定这个样本的类别. 判别函数的形式多种多样,在自然语言处理中,最为常用的判别函数为线性函数。
## 二分类
二分类问题
$$ \hat y =sign((f(z))) = sign(\theta^Tz+\theta_0) $$

用这里的正负号判断

判别函数为 $$ f(z) ＝ \theta^Tz+\theta_0 = \sum_{i=1}^{k}\theta_iz_i + \theta_0 = \sum_{i=0}^{k} = \hat \theta^T \hat z $$ 其中$z_0=1$,$\hat\theta,\hat z$分别称为增广权重向量和增广特征向量。 $$ \hat z = \left( \begin{array} {ccc} 1 \ z_1\ .\ .\ z_k \end{array} \right) = \left( \begin{array}{ccc} 1 \ \ z \ \ \
\end{array} \right) $$

$$ \hat \theta = \left(\begin{array} {ccc} \theta_0 \ \theta_1\ .\ .\ \theta_k \end{array} \right) = \left(\begin{array}{ccc}\theta_0 \ \ \theta \ \ \
\end{array} \right) $$

可以直接用线性回归来进行二分类

在更大数据量下，梯度下降会更好，比最小二乘法更快

In [91]:
#用numpy来进行梯度下降线性回归
import numpy as np
def compute_error_for_line_given_points(b, w, points):
    totalError = 0
    for i in range(len(points)):
        x = points[i, 0]
        y = points[i, 1]
        totalError += (y - (w * x + b)) ** 2
    return totalError / float(len(points)) # average
def step_gradient(b_current, w_current, points, learningRate):
    b_gradient = 0
    w_gradient = 0
    N = float(len(points))
    for i in range(len(points)):
        x = points[i, 0]
        y = points[i, 1]
        b_gradient += 2 * ((w_current * x) + b_current - y)
        w_gradient += 2 * x * ((w_current * x) + b_current - y)
    b_gradient = b_gradient / N
    w_gradient = w_gradient / N
    new_b = b_current - (learningRate * b_gradient)
    new_w = w_current - (learningRate * w_gradient)
    return [new_b, new_w]
def gradient_descent_runner(points, starting_b, starting_w, learning_rate, num_iterations): # num_iteration 迭代次数
    b = starting_b
    w = starting_w
    for i in range(num_iterations):
        b, w = step_gradient(b, w, np.array(points), learning_rate)
    return [b, w]
"""
def run():
    learning_rate = 0.0001
    initial_b = random()
    initial_w = random()
    num_iterations = 1000
    print("Starting gradient descent at b = {0}, w = {1}, error = {2}"
          .format(initial_b, initial_w, 
                  compute_error_for_line_given_points(initial_b, initial_w, points)))
    print("Running...")
    [b, w] = gradient_descent_runner(points, initial_b, initial_w, learning_rate, num_iterations)
    print("After {0} iterations at b = {1}, w = {2}, error = {3}"
          .format(num_iterations, b, w, 
                  compute_error_for_line_given_points(b, w, points)))
run()
"""

'\ndef run():\n    learning_rate = 0.0001\n    initial_b = random()\n    initial_w = random()\n    num_iterations = 1000\n    print("Starting gradient descent at b = {0}, w = {1}, error = {2}"\n          .format(initial_b, initial_w, \n                  compute_error_for_line_given_points(initial_b, initial_w, points)))\n    print("Running...")\n    [b, w] = gradient_descent_runner(points, initial_b, initial_w, learning_rate, num_iterations)\n    print("After {0} iterations at b = {1}, w = {2}, error = {3}"\n          .format(num_iterations, b, w, \n                  compute_error_for_line_given_points(b, w, points)))\nrun()\n'

# 上课笔记

logistic回归
$$y=\frac{1}{1+e^{-(\omega^T+b)}}$$

sigmoid函数 `性质`：函数输出作为概率值;特征加权累加；非线性变化

· x为正例的概率越大，几率取值即越大
· 输出值越接近1说明输入数据x分类为该类别的可能性大
参数求取，损失函数最小化，对数似然的相反数：即交叉熵
$$T(\theta)=log(L(\theta|D))=-\sum{n}{i=1}(y_{i}log(h_{\theta}(x_{i}))+(1-y_{i}log(1-h_{\theta}(x_{i}))$$

从回归到softmax分类：二分类到多分类


矩阵分解的潜在语义分析
奇异值分解：将一个矩阵分解为两个正将矩阵和一个对角矩阵的乘积。

In [ ]:
from sklearn import model_selection
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.externals import joblib
import jieba
import numpy as np

config = LrConfig()


class DataProcess():
    def __init__(self, dataset_path=None, stopwords_path=None, model_save_path=None):
        self.dataset_path = dataset_path
        self.stopwords_path = stopwords_path
        self.model_save_path = model_save_path

    def read_data(self):
        """读取数据"""
        stopwords = list()
        with open(self.dataset_path, encoding='utf-8') as f1:
            data = f1.readlines()
        with open(self.stopwords_path, encoding='utf-8') as f2:
            temp_stopwords = f2.readlines()
        for word in temp_stopwords:
            stopwords.append(word[:-1])
        return data, stopwords

    def save_categories(self, data, save_path):
        """将文本的类别写到本地"""
        with open(save_path, 'w', encoding='utf-8') as f:
            f.write('|'.join(data))

    def pre_data(self, data, stopwords, test_size=0.2):
        """数据预处理"""
        label_list = list()
        text_list = list()
        for line in data:
            label, text = line.split('\t', 1)
            # print(label)
            seg_text = [word for word in jieba.cut(text) if word not in stopwords]
            text_list.append(' '.join(seg_text))
            label_list.append(label)
        # 标签转化为one-hot格式
        encoder_nums = LabelEncoder()
        label_nums = encoder_nums.fit_transform(label_list)
        categories = list(encoder_nums.classes_)
        self.save_categories(categories, config.categories_save_path)
        label_nums = np.array([label_nums]).T
        encoder_one_hot = OneHotEncoder()
        label_one_hot = encoder_one_hot.fit_transform(label_nums)
        label_one_hot = label_one_hot.toarray()
        return model_selection.train_test_split(text_list, label_one_hot, test_size=test_size, random_state=1024)

    # TODO:后续做
    def get_bow(self):
        """提取词袋模型特征"""
        pass

    # TODO:这里可能出现维度过大，内存不足的问题，目前是去除低频词解决，可以做lda或者pca降维（后续做）
    def get_tfidf(self, X_train, X_test):
        """提取tfidf特征"""
        vectorizer = TfidfVectorizer(min_df=100)
        vectorizer.fit_transform(X_train)
        X_train_vec = vectorizer.transform(X_train)
        X_test_vec = vectorizer.transform(X_test)
        return X_train_vec, X_test_vec, vectorizer

    # TODO:后续做
    def get_word2vec(self):
        """提取word2vec特征"""
        pass

    def provide_data(self):
        """提供数据"""
        data, stopwords = self.read_data()
        #  1、提取bag of word参数
        #  2、提取tf-idf特征参数
        X_train, X_test, y_train, y_test = self.pre_data(data, stopwords, test_size=0.2)
        X_train_vec, X_test_vec, vectorizer = self.get_tfidf(X_train, X_test)
        joblib.dump(vectorizer, self.model_save_path)
        #  3、提取word2vec特征参数
        return X_train_vec, X_test_vec, y_train, y_test

    def batch_iter(self, x, y, batch_size=64):
        """迭代器，将数据分批传给模型"""
        data_len = len(x)
        num_batch = int((data_len-1)/batch_size)+1
        indices = np.random.permutation(np.arange(data_len))
        x_shuffle = x[indices]
        y_shuffle = y[indices]
        for i in range(num_batch):
            start_id = i*batch_size
            end_id = min((i+1)*batch_size, data_len)
            yield x_shuffle[start_id: end_id], y_shuffle[start_id: end_id]


In [1]:
import pandas as pd
train=pd.read_csv('G:/download/train/train.tsv',sep='\t')

In [2]:
train['Phrase']

0         A series of escapades demonstrating the adage ...
1         A series of escapades demonstrating the adage ...
2                                                  A series
3                                                         A
4                                                    series
5         of escapades demonstrating the adage that what...
6                                                        of
7         escapades demonstrating the adage that what is...
8                                                 escapades
9         demonstrating the adage that what is good for ...
10                                  demonstrating the adage
11                                            demonstrating
12                                                the adage
13                                                      the
14                                                    adage
15                          that what is good for the goose
16                                      

In [3]:
_Train=train['Phrase']
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
vectorizer = TfidfVectorizer(min_df=100)
a=vectorizer.fit_transform(_Train)


In [71]:
from sklearn.preprocessing import OneHotEncoder
one_h=OneHotEncoder()
sentiment=one_h.fit_transform(spres)

F:\Anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


In [85]:
corpus=_Train.iloc[:2000]

from sklearn.feature_extraction.text import CountVectorizer
vectorizer = TfidfVectorizer()
X_train_vec=vectorizer.fit_transform(corpus)


In [73]:
#提取词袋模型,用countvectorize会产生词频统计
vectorizer = CountVectorizer()

X = vectorizer.fit_transform(corpus)
vectorizer.get_feature_names()



['100',
 '13',
 '25',
 '60',
 'aborted',
 'about',
 'above',
 'absolute',
 'absorbing',
 'accents',
 'across',
 'action',
 'actor',
 'actors',
 'actually',
 'adage',
 'after',
 'against',
 'age',
 'agers',
 'aggressive',
 'aims',
 'airless',
 'all',
 'almost',
 'also',
 'ambition',
 'amounts',
 'amuses',
 'amusing',
 'an',
 'and',
 'any',
 'apart',
 'approach',
 'appropriate',
 'are',
 'arrives',
 'arriving',
 'arthur',
 'arts',
 'as',
 'assault',
 'at',
 'attempts',
 'avengers',
 'avoid',
 'back',
 'bad',
 'banality',
 'bartlett',
 'baseball',
 'be',
 'becomes',
 'being',
 'beneath',
 'best',
 'betrayal',
 'better',
 'beyond',
 'big',
 'bilingual',
 'bind',
 'blame',
 'bloody',
 'boilerplate',
 'both',
 'boy',
 'but',
 'by',
 'bygone',
 'call',
 'camera',
 'can',
 'cartoon',
 'case',
 'cattaneo',
 'celebrated',
 'chabrol',
 'chance',
 'chapter',
 'character',
 'characters',
 'charm',
 'charmer',
 'childhood',
 'chuck',
 'chuckles',
 'cinematic',
 'claude',
 'clearly',
 'clever',
 'cli

In [86]:
X_train_vec

<2000x673 sparse matrix of type '<class 'numpy.float64'>'
	with 8987 stored elements in Compressed Sparse Row format>

In [87]:
from sklearn import svm

clf = svm.SVC( probability=True)
clf.fit(X_train_vec,spres[:2000])


F:\Anaconda3\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
F:\Anaconda3\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
    kernel='rbf', max_iter=-1, probability=True, random_state=None,
    shrinking=True, tol=0.001, verbose=False)

In [88]:
print(clf.predict(X_train_vec[1021]))

[2]


In [89]:
print(spres[1021])

[3]


# 上课笔记 0425
误差向后传播
+ 前馈神经网络
+ 卷积神经网络
+ 循环神经网络

感知机不能解决异或问题

分段学习到端对端学习

+ 神经元 mcp模型

给定n个二值化的输入数据与连接参数 mcp神经元模型对输入数据线性加权求和，然后用函数将加权$\phi(x)$累加结果映射为0或1


# task2 用cnn、lstm实现文本分类任务
+ 要求同task1

In [1]:
import torch
from torch import nn
import numpy as np
from torch.autograd import Variable
import torch.nn.functional as F


class TextRNN(nn.Module):
    """文本分类，RNN模型"""

    def __init__(self):
        super(TextRNN, self).__init__()
        # 三个待输入的数据
        self.embedding = nn.Embedding(5000, 64)  # 进行词嵌入
        self.rnn = nn.LSTM(input_size=64, hidden_size=128, bidirectional=True)
        # self.rnn = nn.GRU(input_size=64, hidden_size=128, num_layers=2, bidirectional=True)
        self.f1 = nn.Sequential(nn.Linear(256, 10),
                                nn.Softmax())

    def forward(self, x):
        x = self.embedding(x) # batch_size x text_len x embedding_size 64*600*64
        x= x.permute(1, 0, 2) # text_len x batch_size x embedding_size 600*64*64
        x, (h_n, c_n)= self.rnn(x) #x为600*64*256, h_n为2*64*128 lstm_out       Sentence_length * Batch_size * (hidden_layers * 2 [bio-direct]) h_n           （num_layers * 2） * Batch_size * hidden_layers
        final_feature_map = F.dropout(h_n, 0.8)
        feature_map = torch.cat([final_feature_map[i, :, :] for i in range(final_feature_map.shape[0])], dim=1) #64*256 Batch_size * (hidden_size * hidden_layers * 2)
        final_out = self.f1(feature_map) #64*10 batch_size * class_num
        return final_out



class TextCNN(nn.Module):
    def __init__(self):
        super(TextCNN, self).__init__()
        self.embedding = nn.Embedding(5000, 64)
        self.conv = nn.Sequential(nn.Conv1d(in_channels=64,
                                        out_channels=256,
                                        kernel_size=5),
                              nn.ReLU(),
                              nn.MaxPool1d(kernel_size=596))

        self.f1 = nn.Linear(256, 10)

    def forward(self, x):
        x = self.embedding(x) # batch_size x text_len x embedding_size 64*600*64
        x = x.permute(0, 2, 1) #64*64*600

        x = self.conv(x)  #Conv1后64*256*596,ReLU后不变,NaxPool1d后64*256*1

        x = x.view(-1, x.size(1)) #64*256
        x = F.dropout(x, 0.8)
        x = self.f1(x)    #64*10 batch_size * class_num
        return x

## 深度学习里的Embedding
这个概念在深度学习领域最原初的切入点是所谓的Manifold Hypothesis（流形假设）。流形假设是指“**自然的原始数据是低维的流形嵌入于(embedded in)原始数据所在的高维空间**”。那么，深度学习的任务就是把高维原始数据（图像，句子）映射到低维流形，使得高维的原始数据被映射到低维流形之后变得可分，而这个映射就叫嵌入（Embedding）。比如Word Embedding，就是把单词组成的句子映射到一个表征向量。但后来不知咋回事，开始把低维流形的表征向量叫做Embedding，其实是一种误用。。。

作者：刘斯坦
链接：https://www.zhihu.com/question/38002635/answer/1382442522
来源：知乎
著作权归作者所有。商业转载请联系作者获得授权，非商业转载请注明出处。

## 理解神经网络中的Dropout
dropout是指在深度学习网络的训练过程中，对于神经网络单元，按照一定的概率将其暂时从网络中丢弃。注意是暂时，对于随机梯度下降来说，由于是随机丢弃，故而每一个mini-batch都在训练不同的网络。

过拟合是深度神经网（DNN）中的一个常见问题：模型只学会在训练集上分类，这些年提出的许多过拟合问题的解决方案，其中dropout具有简单性而且效果也非常良好。


————————————————
版权声明：本文为CSDN博主「SuPhoebe」的原创文章，遵循CC 4.0 BY-SA版权协议，转载请附上原文出处链接及本声明。
原文链接：https://blog.csdn.net/u013007900/article/details/78120669/
## PyTorch 中 LSTM 的 output、h_n 和 c_n 之间的关系
+ h_n：最后一个时间步的输出，即 h_n = output[:, -1, :]（一般可以直接输入到后续的全连接层，在 Keras 中通过设置参数 return_sequences=False 获得）
+ c_n：最后一个时间步 LSTM cell 的状态（一般用不到）


In [2]:
# 读取训练集文件
import sys
from collections import Counter

import numpy as np
import tensorflow.keras as kr
import os

try:
    reload(sys)
    sys.setdefaultencoding("utf-8")
    is_py3 = False
except NameError:
    is_py3 = True


def native_content(content):
    if not is_py3:
        return content.decode('utf-8')
    else:
        return content


def open_file(filename, mode='r'):
    return open(filename, mode, encoding='utf-8', errors='ignore')


def read_file(filename):
    """读取文件数据"""
    contents, labels = [], []
    with open_file(filename) as f:
        for line in f:
            try:
                label, content = line.strip().split('\t')
                if content:
                    contents.append(list(native_content(content)))
                    labels.append(native_content(label))
            except:
                pass
    return contents, labels


def build_vocab(train_dir, vocab_dir, vocab_size=5000):
    """根据训练集构建词汇表，存储"""
    data_train, _ = read_file(train_dir)

    all_data = []
    for content in data_train:
        all_data.extend(content)

    counter = Counter(all_data)
    count_pairs = counter.most_common(vocab_size - 1)  #统计最常出现的字
    # print(count_pairs)
    words, _ = list(zip(*count_pairs))
    # print(words)
    # 添加一个 <PAD> 来将所有文本pad为同一长度
    words = ['<PAD>'] + list(words)
    open_file(vocab_dir, mode='w').write('\n'.join(words) + '\n')


def read_vocab(vocab_dir):
    """读取词汇表"""
    # words = open_file(vocab_dir).read().strip().split('\n')
    with open_file(vocab_dir) as fp:
        # 如果是py2 则每个值都转化为unicode
        words = [native_content(_.strip()) for _ in fp.readlines()]
    word_to_id = dict(zip(words, range(len(words))))
    return words, word_to_id


def read_category():
    """读取分类目录，固定"""
    categories = ['体育', '财经', '房产', '家居', '教育', '科技', '时尚', '时政', '游戏', '娱乐']

    categories = [native_content(x) for x in categories]

    cat_to_id = dict(zip(categories, range(len(categories))))

    return categories, cat_to_id


def to_words(content, words):
    """将id表示的内容转换为文字"""
    return ''.join(words[x] for x in content)


def process_file(filename, word_to_id, cat_to_id, max_length=600):
    """将文件转换为id表示"""
    contents, labels = read_file(filename)

    data_id, label_id = [], []
    for i in range(len(contents)):
        data_id.append([word_to_id[x] for x in contents[i] if x in word_to_id])
        label_id.append(cat_to_id[labels[i]])

    # 使用keras提供的pad_sequences来将文本pad为固定长度
    x_pad = kr.preprocessing.sequence.pad_sequences(data_id, max_length)
    y_pad = kr.utils.to_categorical(label_id, num_classes=len(cat_to_id))  # 将标签转换为one-hot表示

    return x_pad, y_pad


def batch_iter(x, y, batch_size=64):
    """生成批次数据"""
    # print('inter')
    data_len = len(x)
    num_batch = int((data_len - 1) / batch_size) + 1

    indices = np.random.permutation(np.arange(data_len))
    # print(indices)
    x_shuffle = x[indices]
    # print(x_shuffle)
    y_shuffle = y[indices]
    # print(y_shuffle)

    for i in range(num_batch):
        start_id = i * batch_size
        end_id = min((i + 1) * batch_size, data_len)
        yield x_shuffle[start_id:end_id], y_shuffle[start_id:end_id]


## pad_sequences序列预处理
``pad_sequences = tf.contrib.keras.preprocessing.sequence.pad_sequences``

``keras.preprocessing.sequence.pad_sequences(sequences,maxlen=None,dtype='int32',padding='pre',truncating='pre', value=0.)``

解释：

sequences：浮点数或整数构成的两层嵌套列表

maxlen：None或整数，为序列的最大长度。大于此长度的序列将被截短，小于此长度的序列将在后部填0.在命名实体识别任务中，主要是指句子的最大长度

dtype：返回的numpy array的数据类型

padding：‘pre’或‘post’，确定当需要补0时，在序列的起始还是结尾补

truncating：‘pre’或‘post’，确定当需要截断序列时，从起始还是结尾截断

value：浮点数，此值将在填充时代替默认的填充值0

 

返回形如(nb_samples,nb_timesteps)的2D张量


In [3]:
train=read_file("G:\cnews\cnews.train.txt")

In [4]:
build_vocab("G:\cnews\cnews.train.txt","G:\cnews\cnews.vocab.txt")

In [2]:

from __future__ import print_function
import torch
from torch import nn
from torch import optim
from torch.autograd import Variable
import os
def train():
    x_train, y_train = process_file(train_dir, word_to_id, cat_to_id,600)#获取训练数据每个字的id和对应标签的one-hot形式
    x_val, y_val = process_file(val_dir, word_to_id, cat_to_id,600)
    #使用LSTM或者CNN
    model = TextRNN()
    # model = TextCNN()
    #选择损失函数
    #Loss = nn.MultiLabelSoftMarginLoss()
    # Loss = nn.BCELoss()
    Loss = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(),lr=0.001)
    best_val_acc = 0
    for epoch in range(100):
        i = 0
        print('epoch:{}'.format(epoch))
        batch_train = batch_iter(x_train, y_train,64)
        for x_batch, y_batch in batch_train:
            i +=1
            # print(i)
            x = np.array(x_batch)
            y = np.array(y_batch)
            x = torch.LongTensor(x)
            y = torch.Tensor(y)
            # y = torch.LongTensor(y)
            # x = Variable(x)
            # y = Variable(y)
            out = model(x)
            loss = Loss(out,y)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            # 对模型进行验证
            if i % 90 == 0:
                los, accracy = evaluate(model, Loss, x_val, y_val)
                print('loss:{},accracy:{}'.format(los, accracy))
                if accracy > best_val_acc:
                    torch.save(model.state_dict(), 'model_params.pkl')
                    best_val_acc = accracy


In [6]:
base_dir = 'G:\cnews'
train_dir = os.path.join(base_dir, 'cnews.train.txt')
test_dir = os.path.join(base_dir, 'cnews.test.txt')
val_dir = os.path.join(base_dir, 'cnews.val.txt')
vocab_dir = os.path.join(base_dir, 'cnews.vocab.txt')

In [7]:
def evaluate(model, Loss, x_val, y_val):
    """测试集上准确率评估"""
    batch_val = batch_iter(x_val, y_val, 64)
    acc = 0
    los = 0
    for x_batch, y_batch in batch_val:
        size = len(x_batch)
        x = np.array(x_batch)
        y = np.array(y_batch)
        x = torch.LongTensor(x)
        y = torch.Tensor(y)
        # y = torch.LongTensor(y)
        # x = Variable(x)
        # y = Variable(y)
        out = model(x)
        loss = Loss(out, y)
        # optimizer.zero_grad()
        # loss.backward()
        # optimizer.step()
        loss_value = np.mean(loss.detach().numpy())
        accracy = np.mean((torch.argmax(out, 1) == torch.argmax(y, 1)).numpy())
        acc +=accracy*size
        los +=loss_value*size
    return los/len(x_val), acc/len(x_val)

In [ ]:
categories, cat_to_id = read_category()
    #获取训练文本中所有出现过的字及其所对应的id
words, word_to_id = read_vocab(vocab_dir)
    #获取字数
vocab_size = len(words)
print('train')
train()

train
epoch:0


F:\Anaconda3\lib\site-packages\torch\nn\modules\container.py:117: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


loss:0.07750285989046096,accracy:0.3084
loss:0.07362846817970276,accracy:0.3564
loss:0.07446694656610489,accracy:0.3776
loss:0.06720132167339325,accracy:0.4234
loss:0.06628206780552864,accracy:0.4504
loss:0.06595048649311065,accracy:0.4586


https://www.jianshu.com/p/8acb163dd1fe
### python中collection的使用

交叉熵损失函数
$\hat{y}$代表模型预测类别分布概率
+ 对x而言 y和$\hat{y}$的交叉熵损失函数定义为$y×{log(\hat{y})}$
感知机网络
+ 没有隐藏层
+ 无法拟合复杂数据
从标注数据除法优化模型参数

# 上课笔记 0427
## bp算法
将误差从后向前传递，更新参数，将误差分摊给各层所有单元，得到各层单元所产生的误差，根据这个误差让各层单元夫妻各自责任、修正各单元参数。

误差反向传播例子：
$$f(w,x)=\frac{1}{1+e^{-(w_{0}x_{0}+w_{1}x_{1}+w_{2})}}$$
反向传播：
+ 梯度计算
+ 更新参数

机器学习的能力在于拟合和优化
$$ w_{j}^{new}=w_{j}^{old}-\theta×\frac{\partial{Model_{loss}}}{\partial{w_{j}}}$$

## 从BP神经网络到循环神经网络RNN再到LSTM
BP神经网络是线性权重的激活函数模型，即输入一个向量，对向量进行加权处理后输入到隐含层神经元的激活函数当中去，再将函数的输出值进行加权处理最后得到输出层的值。

根据输出层和真实值的关系，我们可以做一个误差函数（用广义的名词来说可以使距离），一般误差函数使用的是均方误差。假设为L。

BP神经网络要优化的参数有不同层之间的权重U，$w_i$  ，V，L是这些权重参数的函数。我们利用L对U，$w_i$ ，V的梯度来对其进行更新，直到L函数不再变动。

RNN（Recurrent Neural Network）是一类用于处理序列数据的神经网络。

基础的神经网络只在层与层之间建立了权连接，RNN最大的不同之处就是在层之间的神经元之间也建立的权连接。




In [1]:
import torch
import torch.nn as nn

x = torch.tensor([2, 3, 4], dtype=torch.float, requires_grad=True)
print(x)
y = x * 2
while y.norm() < 1000:
    y = y * 2
print(y)

y.backward(torch.ones_like(y))
print(x.grad)

tensor([2., 3., 4.], requires_grad=True)
tensor([ 512.,  768., 1024.], grad_fn=<MulBackward0>)
tensor([256., 256., 256.])


In [1]:
import numpy as np
np.array([1,3,5])*np.array([2,4,6])

array([ 2, 12, 30])

# 反向传播算法
假设训练集只有一个实例$(x^{(1)},y^{(1)})$，神经网络是一个四层的神经网络，其中$K=4,S_L=4,L=4$：

前向传播算法： 
 
 <img src='https://i.loli.net/2018/12/02/5c03dbc252db6.png' width=500>
 
我们从最后一层的误差开始计算，误差是激活单元的预测$(a^{(4)}_k)$与实际值$(y^k)$之间的误差，$(k=1:K)$。     

1. 第四层：用δ来表示误差，则：$\delta^{(4)} = a^{(4)} -y$
 
2. 第三层：利用$\delta^{(4)}$来计算前一层的误差：$\delta^{(3)} = (\theta^{(3)})^T\delta^{(4)} * g^`(z^{(3)})$     
 其中$g^`(z^{(3)})$是 S 形函数的导数，$g^`(z^{(3)})=a^{(3)}*(1-a^{(3)})$。而 $(\theta^{(3)})^T\delta^{(4)} $则是权重导致的误差的和。
 
3. 继续计算第二层的误差：$\delta^{(2)} = (\theta^{(2)})^T\delta^{(3)} * g^`(z^{(2)})$ 
4. 第一层是输入变量，不存在误差。

有了所有的误差的表达式后，便可以计算代价函数的偏导数了，假设 $\lambda=0$，即我们不做任何正则化处理时有：
$$\frac{\partial}{\partial\theta^{(l)}_{ij}}j(\theta) = a^{(l)}_{j}\delta^{l+1}_i$$

**重要的是清楚地知道上面式子中上下标的含义：**

+ l 代表目前所计算的是第几层 
+ j 代表目前计算层中的激活单元的下标，也将是下一层的第 j 个输入变量的下标。 
+ i 代表下一层中误差单元的下标，是受到权重矩阵中第 i 行影响的下一层中的误差单元的下标。 

如果我们考虑正则化处理，并且我们的训练集是一个特征矩阵而非向量。在上面的特殊情况中，我们需要计算每一层的误差单元来计算代价函数的偏导数。在更为一般的情况中，我们同样需要计算每一层的误差单元，但是我们需要为整个训练集计算误差单元，此时的误差单元也是一个矩阵，我们用$\Delta^{(l)}_{ij}$来表示这个误差矩阵。第 l 层的第 i 个激活单元受到第 j 个参数影响而导致的误差。 

我们的算法表示为： 
 
  <img src='https://i.loli.net/2018/12/02/5c03df8488f60.png' width=500>
 
即首先用正向传播方法计算出每一层的激活单元，利用训练集的结果与神经网络预测的结果求出最后一层的误差，然后利用该误差运用反向传播法计算出直至第二层的所有误差。
 
在求出了$D^{(l)}_{ij}$之后，我们便可以计算代价函数的偏导数了，计算方法如下： 

 $$\Delta^{(l)}_{ij} := \frac{1}{m}\Delta^{(l)}_{ij}+\lambda\theta^{(l)}_{ij} if j\neq0$$
 $$\Delta^{(l)}_{ij} := \frac{1}{m}\Delta^{(l)}_{ij}+ if j=0$$

In [3]:
#https://blog.csdn.net/lee813/article/details/89609691介绍训练的一个blog
class TwoLayerNet(torch.nn.Module):
    def __init__(self, D_in, H, D_out):
        """
        In the constructor we instantiate two nn.Linear modules and assign them as
        member variables.
        """
        super(TwoLayerNet, self).__init__()
        self.linear1 = torch.nn.Linear(D_in, H)
        self.linear2 = torch.nn.Linear(H, D_out)

    def forward(self, x):
        """
        In the forward function we accept a Tensor of input data and we must return
        a Tensor of output data. We can use Modules defined in the constructor as
        well as arbitrary operators on Tensors.
        """
        h_relu = self.linear1(x).clamp(min=0)
        y_pred = self.linear2(h_relu)
        return y_pred
model = TwoLayerNet(D_in, H, D_out)
criterion = torch.nn.MSELoss(reduction='sum')
optimizer = torch.optim.SGD(model.parameters(), lr=1e-4)
def train():
    for t in range(500):
        # Forward pass: Compute predicted y by passing x to the model
        y_pred = model(x)
        # Compute and print loss
        loss = criterion(y_pred, y) # 计算损失函数
        print(t, loss.item())
        # Zero gradients, perform a backward pass, and update the weights.
        optimizer.zero_grad() # 梯度置零，因为反向传播过程中梯度会累加上一次循环的梯度
        loss.backward() # loss反向传播
        optimizer.step() # 反向传播后参数更新 

NameError: name 'D_in' is not defined

+ 上面一段代码介绍了用`pytorch`训练模型的最基本的使用
+ 在模型编写时候要重写网络结构，编写前向反馈函数
+ 选择损失函数和优化器